In [29]:
from keras.models import load_model
import time
import cv2
import numpy as np
import matplotlib.pyplot as py
import json

In [30]:
model = load_model("MNIST_Numeric_predictor.h5")

In [31]:

#Create a lower and upper bound for what we want to track
#Green color range
lower = np.array([33, 80, 40])
upper = np.array([100, 255, 255])

#Opening is used to remove noise in detected images. 
#It works by filtering out bounderies of detected images, and then dilating them to fill in the bounderies
kernel_open = np.ones((6, 6))

#Closing is the opposite. It dilates and then erodes in order to fill in missing detections within detected images.
kernel_close = np.ones((10,10))

font = cv2.FONT_HERSHEY_SIMPLEX
#We need to create an empty black screen to write our drawn shapes onto. This is so we can recreate something similar to MNIST drawings.
black = np.zeros((480, 640, 3), dtype=np.uint8)
prediction = -1

#Create a list in order to store the detected locations of our green object. 
#This will be used to stitch together a number that is sent to the CNN
draw_points = []

In [33]:
video = cv2.VideoCapture(0)   
while True:
    (grabbed, frame) = video.read()

    if not grabbed:
        video.release()
        break
    
    #flip the frame so that we do not need to draw backwards
    frame = cv2.flip(frame, 1)
    
    #Convert from a blue green red colorspace to hue saturation value colorspace.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    #Apply a masking function using our preset color boundaries. This allows us to retrieve our detected
    mask = cv2.inRange(hsv, lower, upper)
    
    #Apply the closing and opening functions in order to get a clearer detected image.
    #Explanation: https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html
    mask_open = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_open)
    mask_close = cv2.morphologyEx(mask_open,cv2.MORPH_CLOSE,kernel_close)
    
    mask = mask_close
    
    #Get a contour of the detected image. Before this step, we could have applied a threshold function. However, it performed well enough without it.
    #The RETR_EXTERNAL parameter tells the function to just get the outline of the contour.
    #The CHAIN_APPROX_NONE tells the function not to approximate the contour output. We could have used CHAIN_APPROX_SIMPLE, but memory was not an issue.
    (not_needed, contours, not_needed) = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    #Draw a blue square around the immediate object
    cv2.drawContours(frame, contours, -1, (255, 0, 0), 3)
    
    #Draw a nonrotating red rectangle that always encompasses the largest value in the xy dimensions
    for i in range(len(contours)):
        x,y,w,h=cv2.boundingRect(contours[i])
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255), 2)
        cv2.putText(frame, str(i+1),(x,y-10),font,0.55,(0,255,0),1)
    
    #If we have a contour in the image
    if len(contours) >= 1:
        #How to get center of object using moments: 
        #https://www.learnopencv.com/find-center-of-blob-centroid-using-opencv-cpp-python/
        contour = sorted(contours, key = cv2.contourArea)[-1]
        moment = cv2.moments(contour)
        center = (int(moment['m10'] / moment['m00']), int(moment['m01'] / moment['m00']))
        draw_points.append(center)
        
    #If we don't have contours now, but they used to be present.
    #In other words, we hid our object. So we should make a prediction
    elif len(contours) == 0 and len(draw_points) > 0:
        
        #Explanation of gaussian filtered w/ Otsu's thesholding:
        #https://docs.opencv.org/3.4.0/d7/d4d/tutorial_py_thresholding.html
        
        #Grayscale version is used to create a binary contour
        gray = cv2.cvtColor(black, cv2.COLOR_BGR2GRAY)
        gaussian = cv2.GaussianBlur(gray, (5, 5), 0)
        threshold = cv2.threshold(gaussian, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        blackboard_contours = cv2.findContours(threshold.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
       
        #If the blackboard has a contour
        if len(blackboard_contours) >= 1:
            
            contour = sorted(blackboard_contours, key = cv2.contourArea)[-1]
            
            if cv2.contourArea(contour) >= 1500:
                x, y, width, height = cv2.boundingRect(contour)
                number = gray[y : y + height + 50, x : x + width + 50]
                resized = cv2.resize(number, (28, 28))
                array = np.array(resized)
                result = array.astype('float32')/255

                prediction = model.predict(result.reshape(1,28,28, 1))[0]
                prediction = np.argmax(prediction)
                
        #Reset the drawing for the next prediction
        draw_points = []
        black = np.zeros((480, 640, 3), dtype = np.uint8)

    #Display the drawing on the webcam, as well as on the black screen
    for i in range(1, len(draw_points)):
        if (draw_points[i - 1] is not None and draw_points[i] is not None):
            cv2.line(frame, draw_points[i - 1], draw_points[i], (50, 200, 50), 5)
            cv2.line(black, draw_points[i - 1], draw_points[i], (255, 255, 255), 30)
    
    
    cv2.putText(frame, "CNN: " + str(int(prediction)), (10, 50), font,1, (255, 255, 255), 2)
    cv2.imshow("cam",frame)
    
    #Press escape key to close the program
    k = cv2.waitKey(33)
    if k==27:  
        prediction = -1
        break
        
cv2.destroyAllWindows()
video.release()